In [86]:
# Librerias
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Headers para poder acceder a la pagina
header = {
    'User-Agent': 'Chrome 108.0.5359.125',
    'Accept-Language': 'es'
}

# Link de la pagina de Recetas Gratis
url_recetas_gratis = "https://www.recetasgratis.net"
pagina_recetas_gratis = requests.get(url_recetas_gratis, headers=header)
soup_recetas_gratis = BeautifulSoup(pagina_recetas_gratis.content, 'html.parser')

In [87]:
# Funcion para obtener los links de las diferentes categorias que tiene la pagina
def obtener_links_categorias():
    # Obtener los tags <a> de las diferentes categorias
    titulos_categorias_recetas_gratis = []
    contenedores_categorias_recetas_gratis = soup_recetas_gratis.find_all('div', class_="categoria ga")
    for categoria in range(len(contenedores_categorias_recetas_gratis)):
        titulos_categorias_recetas_gratis.append(
            contenedores_categorias_recetas_gratis[categoria].find('a', class_="titulo"))

    # Obtener el link de cada categoría
    links_categorias = []
    for categoria in range(len(titulos_categorias_recetas_gratis)):
        lista_links = list(titulos_categorias_recetas_gratis)[categoria]
        link_categoria_recetasGratis = lista_links.get('href')
        if link_categoria_recetasGratis != "https://www.recetasgratis.net/Recetas-de-Consejos-de-cocina-listado_receta-6146_1.html":
            links_categorias.append(link_categoria_recetasGratis)

    return links_categorias

In [110]:

def obtener_numPaginasPorCategoria(links_categorias):
    final = False
    num_categoria = 0
    links = []

    # Por cada categoria se recogen las diferentes paginas que existen para esa categoria
    for i in links_categorias[9:10]:

        url_prueba = i

        # Comrpobar si ha llegado al final de la lista de links de las categorias
        if num_categoria == len(links_categorias) - 1:
            final = True
        else:
            final = False

        links.append(url_prueba)
        # Bucle que recorre todas las paginas en una categoria, obteniendo los diferentes links de cada hoja
        while (final == False):

            pags_mirar2 = requests.get(url_prueba)
            soup = BeautifulSoup(pags_mirar2.content, 'html.parser')

            # Obtener el marcador de siguiente hoja
            icon_next = soup.find('a', {"class": "next ga"})
            # Si el icono resulta nulo significa que nos encontramos al final de las hojas para esa categoria
            if icon_next is None:
                final = True
                links.append(url_prueba)
            else:
                # Si existe, se obtiene el link de la siguiente hoja
                icon_next = soup.find('a', {"class": "next ga"}).get("href")
                links.append(icon_next)
                url_prueba = icon_next

        num_categoria += 1



    # Navega entre las diferentes hojas de cada categoria y obtiene los links de las diferentes
    nombres_recetas_pag = []
    for i in links:
        url_bs4 = i
        pag_receta = requests.get(url_bs4)
        soup = BeautifulSoup(pag_receta.content, 'html.parser')
        # Guarda los tags <a> pertenecientes al titulo de cada receta por hoja
        informacion = soup.find_all('a', class_='titulo titulo--resultado')

        # Se obtiene el link guardado bajo el titulo de la receta
        for i in range(len(informacion)):
            receta = list(informacion)[i]
            link = receta.get("href")
            # Se aniade a la lista
            nombres_recetas_pag.append(link)

    return nombres_recetas_pag






In [111]:
def obtener_datos_receta(nombres_recetas_pag):
    titulo_recetas = []
    imagen_cadaReceta = []
    recetas_n_comensales = []
    recetas_duracion = []
    recetas_dificultad = []
    recetas_ingredientes = []
    pasos_receta = []

    
    # Recorre cada link de cada receta guardada en la lista
    for j in nombres_recetas_pag:
        link_receta = j
        lista_nueva = []
        pag_receta_info = requests.get(link_receta)
        soup_receta_info = BeautifulSoup(pag_receta_info.content, 'html.parser')

        # Obtener los ingredientes de la receta

        # Algunas recetas resultan ser un top de recetas, por lo que si existe un indice se trata de un top, que en este caso
        # no se tomaran como recetas
        indice = soup_receta_info.find('div', class_="indice")
        if indice is None:

            # Obtener el titulo de la receta
            titulo = soup_receta_info.find('h1', class_="titulo titulo--articulo")
            if titulo is None:
                titulo = "Sin Informacion"
            else:
                titulo = soup_receta_info.find('h1', class_="titulo titulo--articulo").get_text()

            titulo_recetas.append(titulo)

            # Obtener la imagen de la receta
            try:
                imagen_plato_recetasGratis = soup_receta_info.find('div', class_="imagen")

                if imagen_plato_recetasGratis is None:
                    imagen_plato_recetasGratis = "Sin Informacion"
                    imagen_cadaReceta.append(imagen_plato_recetasGratis)
                else:
                    # Se obtiene la etiqueta <img> y de ella la url de la imagen
                    link_imagen_plato = imagen_plato_recetasGratis.find('img').get('src')
                    
                    imagen_cadaReceta.append(link_imagen_plato)
            except Exception as e:
                imagen_plato_recetasGratis = "Sin Informacion"
                imagen_cadaReceta.append(imagen_plato_recetasGratis)

            # Obtener el numero de comensales
            n_comensales = soup_receta_info.find('span', {"class": "property comensales"})
            if n_comensales is None:
                n_comensales = "Sin Informacion"
            else:
                n_comensales = soup_receta_info.find('span', {"class": "property comensales"}).get_text()

            recetas_n_comensales.append(n_comensales)

            # Obtener el tiempo de preparacion de la receta
            tiempo_duracion = soup_receta_info.find('span', {"class": "property duracion"})
            if tiempo_duracion is None:
                tiempo_duracion = "Sin Informacion"
            else:
                tiempo_duracion = soup_receta_info.find('span', {"class": "property duracion"}).get_text()

            recetas_duracion.append(tiempo_duracion)

            # Obtener el nivel de dificultad de la receta
            dificultad_coccion = soup_receta_info.find('span', {"class": "property dificultad"})
            if dificultad_coccion is None:
                dificultad_coccion = "Sin Informacion"
            else:
                dificultad_coccion = soup_receta_info.find('span', {"class": "property dificultad"}).get_text()
            recetas_dificultad.append(dificultad_coccion)

            # Obtener la lista de ingredientes de la receta
            lista_ingredientes = soup_receta_info.find_all('li', class_="ingrediente")

            # De la lista de ingredientes obtener la etiqueta <label> que contiene el texto
            for i in range(len(lista_ingredientes)):
                lista_ingredient = list(lista_ingredientes)[i]
                nombres_ingredientes = lista_ingredient.find_all('label')

                # Se obtiene el texto crudo de todas las etiquetas <label>
                for j in range(len(nombres_ingredientes)):
                    lista_nueva.append(nombres_ingredientes[j].get_text())

            # quitar los saltos de linea
            new_list = []
            for i in lista_nueva:
                new_list.append(i.strip())
            recetas_ingredientes.append(new_list)

            # Obtener los pasos de cada receta
            # En los pasos se debe eliminar el ultimo parrafo ya que no se trata de un paso
            list_temp_pasos = []
            contenedor_pasos = soup_receta_info.find_all('div', class_="apartado")
            for i in range(len(contenedor_pasos)):
                pasos = contenedor_pasos[i].find_all('p')
                for j in pasos:

                    list_temp_pasos.append(j.get_text())
            pasos_receta.append(list_temp_pasos)

    return titulo_recetas, imagen_cadaReceta, recetas_n_comensales, recetas_duracion, recetas_dificultad, recetas_ingredientes, pasos_receta

In [112]:
lista_linksCadaCategoria = obtener_links_categorias()
print(lista_linksCadaCategoria)

['https://www.recetasgratis.net/Recetas-de-Aperitivos-tapas-listado_receta-1_1.html', 'https://www.recetasgratis.net/Recetas-de-Arroces-cereales-listado_receta-9_1.html', 'https://www.recetasgratis.net/Recetas-de-Aves-caza-listado_receta-11_1.html', 'https://www.recetasgratis.net/Recetas-de-Carne-listado_receta-10_1.html', 'https://www.recetasgratis.net/Recetas-de-Cocteles-bebida-listado_receta-15_1.html', 'https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html', 'https://www.recetasgratis.net/Recetas-de-Guisos-Potage-listado_receta-19_1.html', 'https://www.recetasgratis.net/Recetas-de-Huevos-lacteos-listado_receta-18_1.html', 'https://www.recetasgratis.net/Recetas-de-Legumbres-listado_receta-8_1.html', 'https://www.recetasgratis.net/Recetas-de-Mariscos-listado_receta-13_1.html', 'https://www.recetasgratis.net/Recetas-de-Pan-bolleria-listado_receta-16_1.html', 'https://www.recetasgratis.net/Recetas-de-Pasta-listado_receta-5_1.html', 'https://www.recetasgratis.net/Re

In [113]:
lista_paginasCadaCategoria = obtener_numPaginasPorCategoria(lista_linksCadaCategoria)
print(lista_paginasCadaCategoria)

['https://www.recetasgratis.net/articulo-recetas-de-marisco-y-pescado-gallego-76495.html', 'https://www.recetasgratis.net/receta-de-albondigas-de-choco-y-gambas-76053.html', 'https://www.recetasgratis.net/receta-de-gyozas-de-langostinos-75920.html', 'https://www.recetasgratis.net/receta-de-langostinos-congelados-cocidos-75875.html', 'https://www.recetasgratis.net/receta-de-hamburguesa-de-gambas-75799.html', 'https://www.recetasgratis.net/receta-de-hamburguesa-de-marisco-75789.html', 'https://www.recetasgratis.net/receta-de-cazuela-de-calamares-75788.html', 'https://www.recetasgratis.net/receta-de-pimientos-del-piquillo-rellenos-de-langostinos-75704.html', 'https://www.recetasgratis.net/articulo-recetas-de-marisco-para-navidad-75635.html', 'https://www.recetasgratis.net/receta-de-conchitas-a-la-parmesana-75617.html', 'https://www.recetasgratis.net/receta-de-coctel-de-marisco-75544.html', 'https://www.recetasgratis.net/receta-de-chicharron-de-pota-75307.html', 'https://www.recetasgratis.

In [114]:
    titulos, imagen, comensales, duracion, dificultad, ingredientes, pasos = obtener_datos_receta(lista_paginasCadaCategoria)
    df = pd.DataFrame(columns=["titulo", "imagen", "comensales", "duracion", "dificultad", "ingredientes", "pasos"])
    df["titulo"] = titulos
    df["imagen"] = imagen
    df["comensales"] = comensales
    df["duracion"] = duracion
    df["dificultad"] = dificultad
    df["ingredientes"] = ingredientes
    df["pasos"] = pasos


In [115]:
df

,titulo,imagen,comensales,duracion,dificultad,ingredientes,pasos
0,Receta de Albóndigas de choco y gambas,https://cdn0.recetasgratis.net/es/posts/3/5/0/...,4 comensales,1h 30m,Dificultad baja,"[1 cebolla pequeña, 2 dientes de ajo, 2 cuchar...",[Para empezar con la receta de las albóndigas ...
1,Receta de Gyozas de langostinos,https://cdn0.recetasgratis.net/es/posts/0/2/9/...,2 comensales,30m,Dificultad baja,"[100 gramos de langostinos crudos, 1 diente de...",[¿Te preguntas cómo cocinar gyozas? Para empez...
2,Receta de Langostinos congelados cocidos,https://cdn0.recetasgratis.net/es/posts/5/7/8/...,2 comensales,10m,Dificultad baja,"[300 gramos de langostinos congelados enteros,...",[El primer paso consiste en limpiar cuidadosam...
3,Receta de Hamburguesa de gambas,https://cdn0.recetasgratis.net/es/posts/9/9/7/...,2 comensales,30m,Dificultad baja,"[100 gramos de gambas, 1 cebolla chica, 1 huev...",[Para empezar con la receta de hamburguesa de ...
4,Receta de Hamburguesa de marisco,https://cdn0.recetasgratis.net/es/posts/9/8/7/...,3 comensales,15m,Dificultad baja,"[200 gramos de cola de calamar, 1 cebolla chic...",[Para empezar con la receta de hamburguesa de ...
...,...,...,...,...,...,...,...
793,Receta de Pulpos,https://cdn0.recetasgratis.net/es/posts/8/9/3/...,Sin Informacion,Sin Informacion,Sin Informacion,"[1 kilo de pulpo, 3 dientes de ajos, Un vaso d...",[Primeramente troceamos el pulpo en trozos peq...
794,Receta de Calamares rellenos en salsa,https://cdn0.recetasgratis.net/es/posts/1/4/3/...,2 comensales,30m,Dificultad media,"[12 unidades de Calamares pequeños, 1 unidad d...",[Para empezar a preparara estos calamares rell...
795,Receta de Calamares compuestos,https://cdn0.recetasgratis.net/es/posts/0/4/3/...,Sin Informacion,Sin Informacion,Sin Informacion,"[1 kilo de calamares, 1 cebolla, 1 pimento ver...",[Limpiamos los calamares y los cortamos en rod...
796,Receta de Tortilla de camarones,https://cdn0.recetasgratis.net/es/posts/1/5/0/...,4 comensales,1h 30m,Dificultad baja,"[250 gramos de Camarones, 250 gramos de Harina...",[Echas la harina en un bol y poco a poco vas m...


In [116]:
df.to_csv('df_recetas_gratis_9.csv', index=False)

In [118]:
df_recetas_gratis = pd.DataFrame()

csv_unidos = ["df_recetas_gratis_0.csv", "df_recetas_gratis_1.csv", "df_recetas_gratis_2.csv", "df_recetas_gratis_3.csv", "df_recetas_gratis_4.csv", "df_recetas_gratis_5.csv", "df_recetas_gratis_6.csv", "df_recetas_gratis_7.csv", "df_recetas_gratis_8.csv", "df_recetas_gratis_9.csv", "df_recetas_gratis_10.csv", "df_recetas_gratis_11.csv", "df_recetas_gratis_12.csv", "df_recetas_gratis_13.csv", "df_recetas_gratis_14_15.csv", "df_recetas_gratis_16.csv"]

for file in csv_unidos:
    df = pd.read_csv(file)
    df_recetas_gratis = df_recetas_gratis.append(df, ignore_index=True)
    
df_recetas_gratis

,titulo,imagen,comensales,duracion,dificultad,ingredientes,pasos
0,Receta de Ceviche de pescado con mango,https://cdn0.recetasgratis.net/es/posts/0/2/5/...,2 comensales,30m,Dificultad baja,"['2 filetes de pescado tilapia', '2 piezas de ...",['Para hacer la receta de ceviche de pescado c...
1,Receta de Empanadas de bondiola braseada,https://cdn0.recetasgratis.net/es/posts/6/0/5/...,4 comensales,2h 30m,Dificultad media,"['500 gramos de bondiola', '12 discos de empan...",['Para empezar con la preparación de estas emp...
2,Receta de Empanada en freidora de aire,https://cdn0.recetasgratis.net/es/posts/0/0/5/...,2 comensales,15m,Dificultad muy baja,"['6 discos de empanada', '1 huevo', '60 gramos...","['Para empezar, deberás elegir con qué rellena..."
3,Receta de Empanadas árabes abiertas,https://cdn0.recetasgratis.net/es/posts/1/1/5/...,4 comensales,1h 30m,Dificultad baja,"['200 gramos de harina 000', '5 gramos de leva...",['El primer paso de esta receta de empanadas á...
4,Receta de Empanadas de salchicha,https://cdn0.recetasgratis.net/es/posts/5/0/5/...,2 comensales,30m,Dificultad baja,"['3 salchichas', '100 gramos de queso mozzarel...",['El primer paso para esta receta de empanadas...
...,...,...,...,...,...,...,...
20760,Receta de Rollo de espinacas con queso ricotta...,Sin Informacion,Sin Informacion,Sin Informacion,Sin Informacion,"['4-6 cucharadas de queso parmesano rallado', ...",['Precalentar el horno a 200º C Engrasar una b...
20761,Receta de Palta rellena con pollo,https://cdn0.recetasgratis.net/es/posts/5/7/0/...,Sin Informacion,Sin Informacion,Sin Informacion,"['1 palta', '1 pechuga de pollo', '1 zanahoria...","['Sancochar la pechuga.', 'En el mismo caldo d..."
20762,Receta de Alcachofas glaseadas,https://cdn0.recetasgratis.net/es/posts/6/5/0/...,Sin Informacion,Sin Informacion,Sin Informacion,"['6 fondos de alcachofa', '300 gr. de agua min...",['Prepararemos los fondos de alcachofa: Empeza...
20763,Receta de Aguacate y menta,https://cdn0.recetasgratis.net/es/posts/4/0/0/...,Sin Informacion,Sin Informacion,Sin Informacion,"['½ taza de la lechera', '1 taza de leche ente...",['Poner en la licuadora hielo y todos los ingr...


In [119]:
df_recetas_gratis.to_csv('df_recetas_gratis.csv', index=False)